In [3]:
import reg_mri
import os
from glob import glob
from utils import compute_mean_dice
import nibabel as nib
from scipy.spatial.distance import dice
import numpy as np
import itk
import SimpleITK as sitk
import scipy.ndimage
import scipy
import matplotlib.pyplot as plt
from transforms_dict import getRegistrationEvalInverseTransformForMRI, SaveTransformForMRI
from tqdm import tqdm
import monai
import subprocess

In [6]:
def getLog10Jacobian_deformable_ants(warp_img, outfolder, outdataset):
    outdir = os.path.join('output', outdataset, outfolder, 'Jacobian_Deformable')
    try:
        os.mkdir(outdir)
    except:
        pass
    outname = outdir + '/JacobianDeformable_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
    print(outname)
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 0 1"
    subprocess.call(jacobian_command.split(" "))
    ####################
    outdir = os.path.join('output', outdataset, outfolder, 'Log10Jacobian_Deformable')
    try:
        os.mkdir(outdir)
    except:
        pass
    outname = outdir + '/Log10JacobianDeformable_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
    print(outname)
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))

In [2]:
outfolders = [
    "scenario1_feminad_1",
    "scenario1_feminad_8",
    "scenario2_painfact",
    "scenario3_painfact_freeze2",
    "scenario3_painfact_freeze6",
    "scenario3_painfact_freeze7",
    "scenario4_feminad_fold",
    "scenario4_feminad_jac",
    "scenario4_feminad_old_affine",
         ]
models = [
    "scenario-smoothloss/smoothloss_template_local_scenario1_feminad_newmodel_1.0-0.0-1.0.pth",
    "scenario-smoothloss/smoothloss_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0.pth",
    "scenario-smoothloss/smoothloss_local_scenario2_painfact_newmodel_1.0-0.0-8.0.pth",
    "scenario-smoothloss/smoothloss_local_scenario3_painfact_finetune_newmodel_freeze2_1.0-0.0-8.0.pth",
    "scenario-smoothloss/smoothloss_local_scenario3_painfact_finetune_newmodel_freeze6_1.0-0.0-8.0.pth",
    "scenario-smoothloss/smoothloss_local_scenario3_painfact_finetune_newmodel_freeze7_1.0-0.0-8.0.pth",
    "scenario-smoothloss/smoothloss_local_scenario4_feminad_newmodel_fold_1.0-0.0-8.0.pth",
    "scenario-smoothloss/smoothloss_local_scenario4_feminad_newmodel_jac_1.0-0.0-8.0.pth",
    "scenario-smoothloss/smoothloss_local_scenario4_feminad_oldmodel_affinenew_1.0-0.0-8.0.pth",
]

In [8]:
for i, model in enumerate(models):
    outdataset = 'Fakedata'
    outfolder = outfolders[i]
    mris = sorted(glob(os.path.join('dataset2', outdataset, 'MRI_N4_Resample_Norm_Identity_Affine', "*.nii.gz")))
    #atlas_name = "dataset2/Atlas/Identity_Feminad_Template.nii.gz"
    atlas_name = "dataset2/Atlas/Identity_Neatin_MRI_A9.nii.gz"
    affine = nib.load(atlas_name).affine
    header = nib.load(atlas_name).header 

    for i, mri in enumerate(mris):
        outname = os.path.join("output/" + outdataset + "/" + outfolder + "/MRI_N4_Registration_Deformable/" + mri.split('/')[-1].split('.')[0])
        if "newmodel" in model:
            newmodel = True
        else:
            newmodel = False
        newmodel = True
        pred_image, ddfs = reg_mri.main(model, mri, outname, False, True, "local", newmodel=newmodel)
    
        moving_image = ddfs[0].cpu()
    
        #affine_ddf = ddfs[1].cpu()    
        #affine_ddf = affine_ddf.cpu().numpy()        #affine_ddf = np.transpose(affine_ddf, (2, 3, 4, 0, 1))
        #affine_ddf = scipy.ndimage.gaussian_filter(affine_ddf, sigma=1.0) 
    
        deformable_ddf = ddfs[1].cpu()
        deformable_ddf = deformable_ddf.cpu().numpy()
        deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1))
        #deformable_ddf = scipy.ndimage.gaussian_filter(deformable_ddf, sigma=1.0)            
    
        #affine_image = nib.Nifti1Image(affine_ddf, affine, header)
        #outname_affine = "output/Feminad/" + outfolder + "/AffineWarp/" + mri.split('/')[-1].split('.')[0] + ".nii.gz"
        #nib.save(affine_image, outname_affine)
    
        deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)
        outname_deformable = "output/" + outdataset + "/" + outfolder + "/DeformableWarp/" + mri.split('/')[-1].split('.')[0] + ".nii.gz"
        nib.save(deformable_image, outname_deformable)
        
    mridir = 'output/' + outdataset + '/' + outfolder + '/'
    #affine_warps = sorted(glob(os.path.join(mridir, 'AffineWarp', "*.nii.gz")))
    deformable_warps = sorted(glob(os.path.join(mridir, 'DeformableWarp', "*.nii.gz")))
    for i in range(len(deformable_warps)):
    #    #getLog10Jacobian_affine_ants(affine_warps[i], outfolder)    
    #    print(deformable_warps[i])
        getLog10Jacobian_deformable_ants(deformable_warps[i], outfolder, outdataset)  

dataset2/Fakedata/MRI_N4_Resample_Norm_Identity_Affine/0.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/0_MRI.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/0_Resample.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/0_Resample_Reg.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/0_Reg.nii.gz
1.7323993520000158
dataset2/Fakedata/MRI_N4_Resample_Norm_Identity_Affine/1.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/1_MRI.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/1_Resample.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/1_Resample_Reg.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/1_Reg.nii.gz
1.6902685320000046
dataset2/Fakedata/MRI_N4_Resample_Norm_Identity_Affine/10.nii.gz
=> Sa

=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/24_Resample_Reg.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/24_Reg.nii.gz
1.6485237409999627
dataset2/Fakedata/MRI_N4_Resample_Norm_Identity_Affine/25.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/25_MRI.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/25_Resample.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/25_Resample_Reg.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/25_Reg.nii.gz
1.6807537529999763
dataset2/Fakedata/MRI_N4_Resample_Norm_Identity_Affine/26.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/26_MRI.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/26_Resample.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Regis

=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/40_MRI.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/40_Resample.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/40_Resample_Reg.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/40_Reg.nii.gz
1.5946126219999996
dataset2/Fakedata/MRI_N4_Resample_Norm_Identity_Affine/41.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/41_MRI.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/41_Resample.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/41_Resample_Reg.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Registration_Deformable/41_Reg.nii.gz
1.6747974600000362
dataset2/Fakedata/MRI_N4_Resample_Norm_Identity_Affine/42.nii.gz
=> Saved to output/Fakedata/scenario1_feminad_1/MRI_N4_Regis

output/Fakedata/scenario1_feminad_1/Log10Jacobian_Deformable/Log10JacobianDeformable_14.nii.gz.nii.gz
output/Fakedata/scenario1_feminad_1/Jacobian_Deformable/JacobianDeformable_15.nii.gz.nii.gz
output/Fakedata/scenario1_feminad_1/Log10Jacobian_Deformable/Log10JacobianDeformable_15.nii.gz.nii.gz
output/Fakedata/scenario1_feminad_1/Jacobian_Deformable/JacobianDeformable_16.nii.gz.nii.gz
output/Fakedata/scenario1_feminad_1/Log10Jacobian_Deformable/Log10JacobianDeformable_16.nii.gz.nii.gz
output/Fakedata/scenario1_feminad_1/Jacobian_Deformable/JacobianDeformable_17.nii.gz.nii.gz
output/Fakedata/scenario1_feminad_1/Log10Jacobian_Deformable/Log10JacobianDeformable_17.nii.gz.nii.gz
output/Fakedata/scenario1_feminad_1/Jacobian_Deformable/JacobianDeformable_18.nii.gz.nii.gz
output/Fakedata/scenario1_feminad_1/Log10Jacobian_Deformable/Log10JacobianDeformable_18.nii.gz.nii.gz
output/Fakedata/scenario1_feminad_1/Jacobian_Deformable/JacobianDeformable_19.nii.gz.nii.gz
output/Fakedata/scenario1_femi

output/Fakedata/scenario1_feminad_1/Jacobian_Deformable/JacobianDeformable_9.nii.gz.nii.gz
output/Fakedata/scenario1_feminad_1/Log10Jacobian_Deformable/Log10JacobianDeformable_9.nii.gz.nii.gz


...

=> Saved to output/Fakedata/scenario4_feminad_jac/MRI_N4_Registration_Deformable/4_Resample_Reg.nii.gz
=> Saved to output/Fakedata/scenario4_feminad_jac/MRI_N4_Registration_Deformable/4_Reg.nii.gz
1.7060452529999566
dataset2/Fakedata/MRI_N4_Resample_Norm_Identity_Affine/40.nii.gz
=> Saved to output/Fakedata/scenario4_feminad_jac/MRI_N4_Registration_Deformable/40_MRI.nii.gz
=> Saved to output/Fakedata/scenario4_feminad_jac/MRI_N4_Registration_Deformable/40_Resample.nii.gz
=> Saved to output/Fakedata/scenario4_feminad_jac/MRI_N4_Registration_Deformable/40_Resample_Reg.nii.gz
=> Saved to output/Fakedata/scenario4_feminad_jac/MRI_N4_Registration_Deformable/40_Reg.nii.gz
1.671057846000167
dataset2/Fakedata/MRI_N4_Resample_Norm_Identity_Affine/41.nii.gz
=> Saved to output/Fakedata/scenario4_feminad_jac/MRI_N4_Registration_Deformable/41_MRI.nii.gz
=> Saved to output/Fakedata/scenario4_feminad_jac/MRI_N4_Registration_Deformable/41_Resample.nii.gz
=> Saved to output/Fakedata/scenario4_feminad_j

output/Fakedata/scenario4_feminad_jac/Log10Jacobian_Deformable/Log10JacobianDeformable_12.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Jacobian_Deformable/JacobianDeformable_13.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Log10Jacobian_Deformable/Log10JacobianDeformable_13.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Jacobian_Deformable/JacobianDeformable_14.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Log10Jacobian_Deformable/Log10JacobianDeformable_14.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Jacobian_Deformable/JacobianDeformable_15.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Log10Jacobian_Deformable/Log10JacobianDeformable_15.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Jacobian_Deformable/JacobianDeformable_16.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Log10Jacobian_Deformable/Log10JacobianDeformable_16.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Jacobian_Deformable/JacobianDeformable_17.nii.gz.nii.gz
output/Fak

output/Fakedata/scenario4_feminad_jac/Jacobian_Deformable/JacobianDeformable_6.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Log10Jacobian_Deformable/Log10JacobianDeformable_6.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Jacobian_Deformable/JacobianDeformable_7.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Log10Jacobian_Deformable/Log10JacobianDeformable_7.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Jacobian_Deformable/JacobianDeformable_8.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Log10Jacobian_Deformable/Log10JacobianDeformable_8.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Jacobian_Deformable/JacobianDeformable_9.nii.gz.nii.gz
output/Fakedata/scenario4_feminad_jac/Log10Jacobian_Deformable/Log10JacobianDeformable_9.nii.gz.nii.gz


monai.networks.blocks.Warp: Using PyTorch native grid_sample.


RuntimeError: Error(s) in loading state_dict for LocalNet:
	size mismatch for encode_convs.0.0.conv.weight: copying a param with shape torch.Size([16, 2, 7, 7, 7]) from checkpoint, the shape in current model is torch.Size([32, 2, 7, 7, 7]).
	size mismatch for encode_convs.0.0.adn.N.weight: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for encode_convs.0.0.adn.N.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for encode_convs.0.0.adn.N.running_mean: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for encode_convs.0.0.adn.N.running_var: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for encode_convs.0.1.layers.0.conv.weight: copying a param with shape torch.Size([16, 16, 7, 7, 7])

## Compare log10jac with truths

In [48]:
import torch
for outfolder in outfolders:
    distL1s = 0
    distL2s = 0
    for i in range(50):
        log10jac_truth_file = os.path.join("dataset2/Fakedata/Log10Jacobian/JacobianDeformable_" + str(i) + "_Inverse.nii.gz")
        log10jac_pred_file = os.path.join("output/Fakedata/" + outfolder + "/Log10Jacobian_Deformable/Log10JacobianDeformable_" + str(i) + ".nii.gz")
        log10jac_truth = nib.load(log10jac_truth_file).get_fdata()
        log10jac_pred = nib.load(log10jac_pred_file).get_fdata()
        L1 = np.mean(np.abs(log10jac_pred - log10jac_truth))*1000*2
        L2 = np.sqrt(np.sum(np.square(log10jac_pred-log10jac_truth)))
        distL1s += L1
        distL2s += L2    
    distL1s /= 50    
    distL2s /= 50
    print(outfolder)
    print(distL1s)
    print(distL2s)
    print("-"*30)

scenario1_feminad_1
236.51074146352005
241.70335371963927
------------------------------
scenario1_feminad_8
176.12879231820136
148.43883276817735
------------------------------
scenario2_painfact
183.57403281672197
159.08174229513259
------------------------------
scenario3_painfact_freeze2
183.0211682290924
157.47771785068434
------------------------------
scenario3_painfact_freeze6
182.74346359197443
157.13821016517568
------------------------------
scenario3_painfact_freeze7
180.21670991784526
154.580257692407
------------------------------
scenario4_feminad_fold
178.94137085546393
153.21141688876
------------------------------
scenario4_feminad_jac
177.97974855704481
151.59282852860002
------------------------------


FileNotFoundError: No such file or no access: 'output/Fakedata/scenario4_feminad_old_affine/Log10Jacobian_Deformable/Log10JacobianDeformable_0.nii.gz'

## Compare ddf with truths

In [49]:
import torch
for outfolder in outfolders:
    distL1s = 0
    distL2s = 0
    for i in range(50):
        ddf_truth_file = os.path.join("dataset2/Fakedata/DeformableWarp/" + str(i) + "_InverseWarp.nii.gz")
        ddf_pred_file = os.path.join("output/Fakedata/" + outfolder + "/DeformableWarp/" + str(i) + ".nii.gz")
        ddf_truth = nib.load(ddf_truth_file).get_fdata()
        ddf_pred = nib.load(ddf_pred_file).get_fdata()
        L1 = np.mean(np.abs(ddf_pred - ddf_truth))*1000*2
        L2 = np.sqrt(np.sum(np.square(ddf_pred-ddf_truth)))
        distL1s += L1
        distL2s += L2    
    distL1s /= 50    
    distL2s /= 50
    print(outfolder)
    print(distL1s)
    print(distL2s)
    print("-"*30)

scenario1_feminad_1
3722.0481611430278
5800.8665910467225
------------------------------
scenario1_feminad_8
3685.589638348227
5711.758207178297
------------------------------
scenario2_painfact
3855.1284024802717
6138.4379284578245
------------------------------
scenario3_painfact_freeze2
4029.416845519265
6400.837135892315
------------------------------
scenario3_painfact_freeze6
3841.1872330038727
6123.917179606011
------------------------------
scenario3_painfact_freeze7
3742.4568822669457
5963.640994001288
------------------------------
scenario4_feminad_fold
3693.039919524497
5743.968255537733
------------------------------
scenario4_feminad_jac
3730.1151420944334
5794.046530986766
------------------------------


FileNotFoundError: No such file or no access: 'output/Fakedata/scenario4_feminad_old_affine/DeformableWarp/0.nii.gz'